In [ ]:
!pip install iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import pandas as pd 
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import VarianceThreshold

from sklearn.decomposition import PCA

import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_df = pd.read_csv('train_features.csv')
target_df = pd.read_csv('train_targets_scored.csv')
test_df = pd.read_csv('test_features.csv')
drug_df = pd.read_csv('train_drug.csv')


In [ ]:
#make sure everything is in order
assert (list(train_df['sig_id']) == list(target_df['sig_id']))
sig_id = list(train_df['sig_id'])

In [ ]:
#create dummy variables for categorical features and drop sig_id
def dumb(df):
    df['cp_type'] = df['cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    df = df.drop(['sig_id'],axis = 1)
    return df


train_df = dumb(train_df)
test_df = dumb(test_df)
#target_df = target_df.drop(['sig_id'],axis=1)

In [ ]:
#Remove control subjects. They have zero MoA and are given drug cacb2b860 (DMSO)
target_df = target_df.loc[train_df['cp_type'] == 0].reset_index(drop=True) #also reflect in the target matrix
drug_df = drug_df.loc[train_df['cp_type'] == 0].reset_index(drop=True)
train_df = train_df.loc[train_df['cp_type'] == 0].reset_index(drop=True)

In [ ]:
#add PCA as features to train and testing set
# pca = PCA(n_components = 256)
# pca_array = pca.fit_transform(train_df.to_numpy())
# train_df = pd.concat([train_df,pd.DataFrame(pca_array,columns = ['PC' + str(i) for i in range(pca_array.shape[1])])],axis=1)

In [ ]:
from torch.utils.data.dataloader import T
#selection of top features
vt_fct = VarianceThreshold(threshold=0.8)
vt_fct.fit(train_df.to_numpy()[:,3:]) #exclude categorical features

temp = vt_fct.get_support(indices = True)
temp = temp + 3
temp = np.copy(np.concatenate([np.array([1,2]), temp]))
top_features = temp.tolist()


**HYPERPARAMETERS:**
- number of epochs
- dropout of each layer
- number of nodes
- number of hidden layers
- 

In [ ]:
def create_folds(num_start,num_splits, threshold):
    '''
    implementation of k-fold cross validation and stratification of drugs without much targets
    next step - remove drugs that have little targets?

    theshold for target = 16? how to best choose threshold?
    '''
    folds = []

    # LOAD FILES
    train_df = pd.read_csv('train_features.csv')
    score_df = pd.read_csv('train_targets_scored.csv')
    drug_df = pd.read_csv('train_drug.csv')
    #remove controls
    score_df = score_df.loc[train_df['cp_type'] == 'trt_cp',:]
    drug_df = drug_df.loc[train_df['cp_type'] == 'trt_cp',:]
    targets = score_df.columns[1:] #remove sig_id
    score_df = score_df.merge(drug_df,on = 'sig_id', how = 'left')

    drug_counts = score_df['drug_id'].value_counts()

    below_threshold = drug_counts.loc[drug_counts <= threshold].index.sort_values()
    above_threshold = drug_counts.loc[drug_counts > threshold].index.sort_values()

    # train_feats = pd.read_csv('train_features.csv')
    # scored = pd.read_csv('train_targets_scored.csv')
    # drug = pd.read_csv('train_drug.csv')
    # scored = scored.loc[train_feats['cp_type'] == 'trt_cp', :]
    # drug = drug.loc[train_feats['cp_type'] == 'trt_cp', :]
    # targets = scored.columns[1:]
    # scored = scored.merge(drug, on='sig_id', how='left') 

    #find drug counts for each moa
    #score_df = target_df.merge(drug_df, on = 'sig_id', how = 'left')
        

    for seed in range(num_start):

        dict_b = {} #below_threshold
        dict_a = {} #above threshold
        #stratification of drugs below threshold
        xval_model = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        temp = score_df.groupby('drug_id')[targets].mean().loc[below_threshold]

        for fold,(T,V) in enumerate(xval_model.split(temp,temp[targets])):
            dict_append = {k:fold for k in temp.index[V].values}
            dict_b.update(dict_append)

        #stratification of drugs above threshold
        xval_model = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        temp = score_df.loc[score_df['drug_id'].isin(above_threshold)].reset_index(drop = True)
        for fold,(T,V) in enumerate(xval_model.split(temp,temp[targets])):
            dict_append = {k:fold for k in temp['sig_id'][V].values}
            dict_a.update(dict_append)

        #assign folds
        score_df['fold'] = score_df['drug_id'].map(dict_b)
        score_df.loc[score_df['fold'].isna(),'fold'] = score_df.loc[score_df['fold'].isna(),'sig_id'].map(dict_a)
        score_df['fold'] = score_df['fold'].astype('int8')
        folds.append(score_df['fold'].values)

        del score_df['fold']

    return np.stack(folds)
        


In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
pca_array = pca.fit_transform(train_df.to_numpy())

In [ ]:
# for i,col_name in enumerate(['PC' + str(i+1) for i in range(16)]):
#     train_df[col_name] = pca_array[:,i]

In [ ]:
n_fold = 4
n_start = 4
n_epoch = 16
threshold = 16
random_state = 42 #the meaning of life

batch_size = 512
val_batch_size = batch_size * 4

n_targets = target_df.shape[1] -1 
targets = [col for col in target_df.columns][1:]

criterion = nn.BCELoss()  # Binary Cross Entropy loss function

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

folds_cv = create_folds(n_start, n_fold, threshold)


In [ ]:
folds_cv

In [ ]:
unique, counts = np.unique(folds_cv[0], return_counts=True)
for i in unique:
    print(i, counts[i])
del unique
del counts

In [ ]:
class MoaModel(nn.Module):
    def __init__(self, num_columns):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.2) #train hyperparameter 
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024, 1024))

        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, 206))
        
        # self.batch_norm4 = nn.BatchNorm1d(1024)
        # self.dropout4 = nn.Dropout(0.2)
        # self.dense4 = nn.utils.weight_norm(nn.Linear(1024, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.sigmoid(self.dense3(x))
        
        # x = self.batch_norm4(x)
        # x = self.dropout4(x)
        # x = F.sigmoid(self.dense4(x))
        
        return x

In [ ]:
#create space for hyperparameter, input is model class, amazon sagemaker for choice of hyperparameter
#grid search, random search, bayesian search, 

In [ ]:
class MoaDataset(Dataset):
    def __init__(self, df, targets, feats_idx, mode='train'):
        self.mode = mode
        self.feats = feats_idx
        self.data = df[:, feats_idx]
        if mode=='train':
            self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            return torch.FloatTensor(self.data[idx]), torch.FloatTensor(self.targets[idx])
        elif self.mode == 'test':
            return torch.FloatTensor(self.data[idx]), 0

In [ ]:
loss_array = np.zeros((n_start,n_epoch))
val_array = np.zeros((n_start,n_epoch))

counter = 0
for seed in range(n_start):
    
    print(f'Train seed {seed}')
    set_seed(seed)
    for fold in range(n_fold):
        tr_idx = folds_cv[seed] != fold
        te_idx = folds_cv[seed] == fold

        xtrain = train_df.to_numpy()[tr_idx]
        ytrain = target_df.drop(['sig_id'],axis=1).to_numpy()[tr_idx]
        xval = train_df.to_numpy()[te_idx]
        yval = target_df.drop(['sig_id'],axis=1).to_numpy()[te_idx]

        #print(ytrain)

        train_set = MoaDataset(xtrain, ytrain, top_features)
        val_set = MoaDataset(xval, yval, top_features)
        
        dataloaders = {
            'train': DataLoader(train_set, batch_size=batch_size, shuffle=True),
            'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False)
        }

        model = MoaModel(len(top_features)).to(device)
        checkpoint_path = f'repeat:{seed}_Fold:{fold+1}.pt'


        optimizer = optim.AdamW(model.parameters(), weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                         factor=0.1, patience=3, 
                                                         eps=1e-4, verbose=True)
        best_loss = {'train': np.inf, 'val': np.inf}
        
        for epoch in range(n_epoch):
            epoch_loss = {'train': 0.0, 'val': 0.0}
          
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()
               
                running_loss = 0.0
                
                for i, (x, y) in enumerate(dataloaders[phase]):
                    x, y = x.to(device), y.to(device)
                    
                    optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase=='train'):
                        preds = model(x)
                        loss = criterion(preds, y)

                        
                        if phase=='train':
                            loss.backward()
                            optimizer.step()
                        
                    running_loss += loss.item() / len(dataloaders[phase])
                    


                epoch_loss[phase] = running_loss

        
            print("Epoch {}/{}   -   loss: {:5.5f}   -   val_loss: {:5.5f}".format(epoch+1, n_epoch, epoch_loss['train'], epoch_loss['val']))

            loss_array[counter,epoch] = epoch_loss['train']
            val_array[counter,epoch] = epoch_loss['val']
            
            scheduler.step(epoch_loss['val'])
            
            if epoch_loss['val'] < best_loss['val']:
                best_loss = epoch_loss
                torch.save(model.state_dict(), checkpoint_path)


    
    counter += 1



In [ ]:
#initiate matrices holding data
test_np = test_df.to_numpy()

oof = np.zeros((len(train_df.to_numpy()), n_start, n_targets))
oof_targets = np.zeros((len(train_df.to_numpy()), n_targets))
preds = np.zeros((len(test_np), n_targets))

In [ ]:
def mean_log_loss(y_true, y_pred):
    metrics = []
    worst_target = None
    worst_loss = 0.
    all_targets_ll = {}
    for i, target in enumerate(targets):
        _ll = log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1])
        metrics.append(_ll)
        all_targets_ll[target] = _ll
        if _ll > worst_loss:
            worst_loss = _ll
            worst_target = target
    return np.mean(metrics), (worst_target, worst_loss), all_targets_ll

In [ ]:
worst_targets_in_seed = []
all_targets_ll_in_seed = []
for seed in range(n_start):
    print(f"Inference for seed {seed}")
    seed_targets = []
    seed_oof = []
    seed_preds = np.zeros((len(test_np), n_targets, n_fold))
    print(seed_preds.shape)
    
    for fold in range(n_fold):
        # print(n, len(tr))
        tr_idx = folds_cv[seed] != fold
        te_idx = folds_cv[seed] == fold
        xval = train_df.to_numpy()[te_idx]
        yval = target_df.drop(['sig_id'],axis=1).to_numpy()[te_idx]
        fold_preds = []
        
        val_set = MoaDataset(xval, yval, top_features)
        test_set = MoaDataset(test_np, None, top_features, mode='test')
        
        dataloaders = {
            'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False),
            'test': DataLoader(test_set, batch_size=val_batch_size, shuffle=False)
        }
        
        checkpoint_path = f'repeat:{seed}_Fold:{fold+1}.pt'
        model = MoaModel(len(top_features)).to(device)
        model.load_state_dict(torch.load(checkpoint_path))
        model.eval()
        
        for phase in ['val', 'test']:
            for i, (x, y) in enumerate(dataloaders[phase]):
                # print(i)
                if phase == 'val':
                    x, y = x.to(device), y.to(device)
                elif phase == 'test':
                    x = x.to(device)
                
                with torch.no_grad():
                    batch_preds = model(x)
                    
                    if phase == 'val':
                        seed_targets.append(y)
                        seed_oof.append(batch_preds)
                        # print(y_pred_lr.shape)
                    elif phase == 'test':
                        fold_preds.append(batch_preds)
                    
        fold_preds = torch.cat(fold_preds, dim=0).cpu().numpy()
        seed_preds[:, :, fold] = fold_preds
        
    seed_targets = torch.cat(seed_targets, dim=0).cpu().numpy()
    seed_oof = torch.cat(seed_oof, dim=0).cpu().numpy()
    seed_preds = np.mean(seed_preds, axis=2)
    
    #print("Score for this seed {:5.5f}".format(mean_log_loss(seed_targets, seed_oof)[0]))
    worst_targets_in_seed.append(mean_log_loss(seed_targets, seed_oof)[1])
    all_targets_ll_in_seed.append(mean_log_loss(seed_targets, seed_oof)[2])
    oof_targets = seed_targets
    oof[:, seed, :] = seed_oof
    preds += seed_preds / n_start

oof = np.mean(oof, axis=1)
print("Overall score is {:5.5f}".format(mean_log_loss(oof_targets, oof)[0]))

In [ ]:
ss = pd.read_csv('sample_submission.csv')
ss[targets] = preds
ss.loc[test_df['cp_type']== 1, targets] = 0
ss.to_csv('submission.csv', index=False)